In [1]:
# import dependencies
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
from splinter import Browser
import pandas as pd

In [2]:
# scrape draft results
qbURL = "https://nflcombineresults.com/nflcombinedata.php?year=all&pos=QB&college="
scrapedDF = pd.read_html(qbURL)[0]
scrapedDF.head()

,Year,Name,College,POS,Height (in),Weight (lbs),Wonderlic,40 Yard,Bench Press,Vert Leap (in),Broad Jump (in),Shuttle,3Cone,Unnamed: 13,Unnamed: 14
0,2020.0,Kelly Bryant,Missouri,QB,75.0,225.0,NaN,4.69,NaN,35.0,125.0,4.51,7.33,NaN,NaN
1,2020.0,Joe Burrow,Louisiana State,QB,76.0,216.0,NaN,9.99,NaN,NaN,NaN,9.99,9.99,NaN,NaN
2,2020.0,Kevin Davidson,Princeton,QB,76.0,225.0,NaN,5.00,NaN,28.0,109.0,4.37,7.13,NaN,NaN
3,2020.0,Jacob Eason,Washington,QB,78.0,227.0,NaN,4.89,NaN,27.5,110.0,4.75,7.50,NaN,NaN
4,2020.0,Jake Fromm,Georgia,QB,74.0,220.0,NaN,5.01,NaN,30.0,111.0,4.51,7.27,NaN,NaN


In [3]:
# process data
scrapedDF = scrapedDF.rename(columns={'Year':'year','Name':'name','College':'college','POS':'position',\
                                      'Height (in)':'height','Weight (lbs)':'weight','40 Yard':'40yard',
                                      'Bench Press':'benchpress','Vert Leap (in)':'vertleap',
                                      'Broad Jump (in)':'broadjump','Shuttle':'shuttle','3Cone':'3cone'})
draftDF = scrapedDF[['year','name','college','position','height','weight','40yard','benchpress','vertleap','broadjump',\
                     'shuttle','3cone']]
draftDF = draftDF.dropna(subset=['name'])
draftDF

,year,name,college,position,height,weight,40yard,benchpress,vertleap,broadjump,shuttle,3cone
0,2020.0,Kelly Bryant,Missouri,QB,75.0,225.0,4.69,NaN,35.0,125.0,4.51,7.33
1,2020.0,Joe Burrow,Louisiana State,QB,76.0,216.0,9.99,NaN,NaN,NaN,9.99,9.99
2,2020.0,Kevin Davidson,Princeton,QB,76.0,225.0,5.00,NaN,28.0,109.0,4.37,7.13
3,2020.0,Jacob Eason,Washington,QB,78.0,227.0,4.89,NaN,27.5,110.0,4.75,7.50
4,2020.0,Jake Fromm,Georgia,QB,74.0,220.0,5.01,NaN,30.0,111.0,4.51,7.27
...,...,...,...,...,...,...,...,...,...,...,...,...
708,1987.0,Loren Snyder,Northern Colorado,QB,75.6,207.0,5.15,6.0,26.5,104.0,4.63,9.99
709,1987.0,Kelly Stouffer,Colorado State,QB,75.0,212.0,4.95,NaN,28.0,108.0,4.51,9.99
710,1987.0,Kevin Sweeney,Frenso State (CA),QB,71.3,187.0,9.99,NaN,NaN,NaN,9.99,9.99
711,1987.0,Vinny Testaverde,Miami (FL),QB,76.3,213.0,4.75,13.0,NaN,NaN,9.99,9.99


In [4]:
# commit to sqlite database
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
engine = create_engine('sqlite:///../data/fbdata.sqlite')
draftDF.to_sql('combineStats', con=engine, if_exists='replace', index=False)